# Metadata

**L1 Taxonomy** - Backend Integration

**L2 Taxonomy** - Webhooks

**Subtopic** - Simulating network delays and failures to test webhook resilience

**Use Case** - Develop a Python script that simulates network delays and failures. The script should include a simple webhook server using a lightweight HTTP server from the Python standard library. It should also include a client that sends requests to the server. The server should be designed to randomly delay responses or fail to respond to mimic real-world network conditions. The client should be designed to handle these conditions gracefully, with appropriate error handling and retry logic.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

## Problem Overview

You are tasked with building a **modular Python system** that simulates real-world network delays and failures in webhook communication. The system should consist of two key components:

1. A **Webhook Server** that listens for incoming HTTP POST requests and randomly delays or drops responses to mimic unstable network behavior.
2. A **Webhook Client** that sends POST requests to the server with retry logic, exponential backoff, and error handling for timeouts, dropped connections, and non-200 responses.

The goal is to develop an independent, testable Python module that allows integration with other systems or automated testing pipelines. Your solution must simulate unpredictable network conditions and ensure that the client behaves resiliently under those conditions.


## Input Format

* There is **no standard input**. Instead, the script accepts the following via function arguments or configuration files:

  * **Server parameters** such as port number and failure/delay probabilities.
  * **Client configuration**, including:

    * URL of the webhook server.
    * Retry count.
    * Backoff factor.
    * Timeout per request.
    * JSON payload to send.


## Output Format

* Server output: Logs to the console when a request is received, delayed, responded to, or dropped.
* Client output: Logs to the console detailing:

  * Attempt number.
  * Whether the request succeeded or failed.
  * Error messages (e.g., Timeout, ConnectionError).
  * Time of retry if applicable.

All outputs must be printed to the standard output in human-readable form, suitable for debugging and monitoring.


## Examples

**Example 1:**

```plaintext
[Server] Listening on port 8080
[Server] Received request from client. Delaying for 3.2s...
[Server] Responded with 200 OK

[Client] Attempt 1: TimeoutError - retrying in 1.0s
[Client] Attempt 2: Success - Status Code 200
```

**Example 2:**

```plaintext
[Server] Dropping request (simulated failure)

[Client] Attempt 1: ConnectionError - retrying in 2.0s
[Client] Attempt 2: TimeoutError - retrying in 4.0s
[Client] Attempt 3: Failed - Max retries reached
```


# Requirements

## Explicit Requirements

* The server must:

  * Randomly delay responses by a configurable maximum (e.g., up to 5 seconds).
  * Randomly drop requests based on a failure probability (e.g., 30%).
  * Use only Python's built-in libraries (e.g., `http.server`, `random`, `time`, `threading`).

* The client must:

  * Support a configurable number of retries.
  * Implement exponential backoff with jitter.
  * Handle HTTP timeouts, dropped connections, and non-200 status codes gracefully.
  * Send a JSON payload with each request using only Python's built-in libraries (e.g., `http.client`, `urllib`, `json`).


## Implicit Requirements

* The server and client must run in separate threads or processes to simulate real interaction.
* The client must not crash on unexpected errors.
* Code should be modular and reusable (i.e., server and client should be implemented as classes or functions in a standalone module).
* Avoid using third-party libraries (e.g., `requests`, `flask`, etc.).
* All logs should include timestamps for better observability.


## Function Signature(s) and/or Class Definition(s)

```python

class WebhookServer:
    def __init__(self, host: str, port: int, drop_probability: float, max_delay: float):
        ...
    
    def start(self):
        """Starts the HTTP server in a blocking or threaded mode."""
        ...
```

```python

class WebhookClient:
    def __init__(self, url: str, retries: int, backoff_factor: float, timeout: float):
        ...
    
    def send_payload(self, payload: Dict[str, Any]):
        """Attempts to send the given JSON payload to the configured URL."""
        ...
```


## Edge Cases

* Server configured with 100% drop probability (no responses at all).
* Client with 0 retries (must fail after first error).
* Delay exceeds client timeout (client must retry).
* Payload is malformed (client must handle serialization errors).
* Server receives a request while already processing a delayed response.


## Constraints

* Python version must be 3.7 or above.
* Solution must use only the Python Standard Library.
* Maximum server delay configurable up to 10 seconds.
* Drop probability must be a float between 0.0 and 1.0.
* Backoff factor must be a positive float; retries must be a non-negative integer.
* The solution must be self-contained and easy to run from the command line using:

```bash
python simulate_network_conditions.py --mode server
python simulate_network_conditions.py --mode client
```

In [ ]:
# code

"""
Webhook Network Simulation System

This module simulates real-world network delays and failures in webhook
communication. It consists of a WebhookServer that randomly delays or drops
responses, and a WebhookClient with retry logic and exponential backoff.

Usage:
    python simulate_network_conditions.py --mode server
    python simulate_network_conditions.py --mode client
"""

import argparse
import json
import random
import time
import threading
from datetime import datetime
from http.server import HTTPServer, ThreadingHTTPServer, BaseHTTPRequestHandler
from http.client import HTTPConnection, HTTPSConnection
from typing import Dict, Any, Optional
from urllib.parse import urlparse
import socket
import sys


class TimestampedLogger:
    """A simple logger that adds timestamps to all messages."""

    @staticmethod
    def log(message: str) -> None:
        """Log a message with timestamp to stdout.

        Args:
            message: The message to log
        """
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        print(f"[{timestamp}] {message}")


class WebhookRequestHandler(BaseHTTPRequestHandler):
    """HTTP request handler for the webhook server."""

    def do_POST(self) -> None:
        """Handle POST requests with random delays and drops."""
        client_address = self.client_address[0]

        # Check if we should drop this request
        if random.random() < self.server.drop_probability:
            TimestampedLogger.log(
                f"[Server] Dropping request from {client_address} "
                f"(simulated failure)"
            )
            # Close connection without response to simulate dropped request
            self.connection.close()
            return

        # Read request body
        content_length = int(self.headers.get('Content-Length', 0))
        if content_length > 0:
            body = self.rfile.read(content_length)
            try:
                json.loads(body.decode('utf-8'))
                TimestampedLogger.log(
                    f"[Server] Received request from {client_address}"
                )
            except (json.JSONDecodeError, UnicodeDecodeError):
                TimestampedLogger.log(
                    f"[Server] Received malformed request from "
                    f"{client_address}"
                )
        else:
            TimestampedLogger.log(
                f"[Server] Received empty request from {client_address}"
            )

        # Apply random delay
        delay = random.uniform(0, self.server.max_delay)
        if delay > 0:
            TimestampedLogger.log(
                f"[Server] Delaying response for {delay:.1f}s..."
            )
            time.sleep(delay)

        # Send successful response
        self.send_response(200)
        self.send_header('Content-Type', 'application/json')
        self.end_headers()

        response = {"status": "success", "timestamp": time.time()}
        self.wfile.write(json.dumps(response).encode('utf-8'))

        TimestampedLogger.log("[Server] Responded with 200 OK")


class WebhookServer:
    """HTTP server that simulates network instability with delays and drops."""

    def __init__(self, host: str, port: int, drop_probability: float,
                 max_delay: float) -> None:
        """Initialize the webhook server.

        Args:
            host: The host address to bind to
            port: The port number to listen on
            drop_probability: Probability (0.0-1.0) of dropping requests
            max_delay: Maximum delay in seconds for responses

        Raises:
            ValueError: If parameters are out of valid ranges
        """
        if not 0.0 <= drop_probability <= 1.0:
            raise ValueError("drop_probability must be between 0.0 and 1.0")
        if max_delay < 0 or max_delay > 10:
            raise ValueError("max_delay must be between 0 and 10 seconds")

        self.host = host
        self.port = port
        self.drop_probability = drop_probability
        self.max_delay = max_delay
        self.server: Optional[HTTPServer] = None
        self.server_thread: Optional[threading.Thread] = None

    def start(self, threaded: bool = False) -> None:
        """Start the HTTP server in blocking or threaded mode.

        Args:
            threaded: If True, start server in a separate thread
        """
        try:
            # Use ThreadingHTTPServer for concurrent request handling
            self.server = ThreadingHTTPServer((self.host, self.port),
                                            WebhookRequestHandler)
            # Pass configuration to request handler
            self.server.drop_probability = self.drop_probability
            self.server.max_delay = self.max_delay

            TimestampedLogger.log(
                f"[Server] Starting webhook server on {self.host}:"
                f"{self.port}"
            )
            TimestampedLogger.log(
                f"[Server] Drop probability: {self.drop_probability:.1%}, "
                f"Max delay: {self.max_delay}s"
            )

            if threaded:
                self.server_thread = threading.Thread(
                    target=self.server.serve_forever
                )
                self.server_thread.daemon = True
                self.server_thread.start()
                TimestampedLogger.log("[Server] Server started in background")
            else:
                TimestampedLogger.log(
                    f"[Server] Listening on port {self.port}"
                )
                self.server.serve_forever()

        except OSError as e:
            TimestampedLogger.log(f"[Server] Failed to start server: {e}")
            raise

    def stop(self) -> None:
        """Stop the server gracefully."""
        if self.server:
            TimestampedLogger.log("[Server] Shutting down server...")
            self.server.shutdown()
            self.server.server_close()

            if self.server_thread:
                self.server_thread.join(timeout=5)


class WebhookClient:
    """HTTP client with retry logic and exponential backoff."""

    def __init__(self, url: str, retries: int, backoff_factor: float,
                 timeout: float) -> None:
        """Initialize the webhook client.

        Args:
            url: The target webhook URL
            retries: Number of retry attempts
            backoff_factor: Exponential backoff multiplier
            timeout: Request timeout in seconds

        Raises:
            ValueError: If parameters are invalid
        """
        if retries < 0:
            raise ValueError("retries must be non-negative")
        if backoff_factor <= 0:
            raise ValueError("backoff_factor must be positive")
        if timeout <= 0:
            raise ValueError("timeout must be positive")

        self.url = url
        self.retries = retries
        self.backoff_factor = backoff_factor
        self.timeout = timeout

        # Parse URL for connection details
        parsed = urlparse(url)
        self.scheme = parsed.scheme
        self.host = parsed.hostname
        self.port = parsed.port
        self.path = parsed.path or '/'

        if not self.host:
            raise ValueError("Invalid URL: missing hostname")

        # Set default ports
        if not self.port:
            self.port = 443 if self.scheme == 'https' else 80

    def _create_connection(self) -> HTTPConnection:
        """Create an HTTP connection based on URL scheme.

        Returns:
            HTTPConnection or HTTPSConnection instance
        """
        if self.scheme == 'https':
            return HTTPSConnection(self.host, self.port, timeout=self.timeout)
        else:
            return HTTPConnection(self.host, self.port, timeout=self.timeout)

    def _send_request(self, payload: Dict[str, Any]) -> bool:
        """Send a single HTTP request.

        Args:
            payload: JSON payload to send

        Returns:
            True if request succeeded, False otherwise

        Raises:
            Various exceptions for different failure types
        """
        conn = None
        try:
            # Serialize payload
            json_data = json.dumps(payload)

            # Create connection
            conn = self._create_connection()
            conn.connect()

            # Send request
            headers = {
                'Content-Type': 'application/json',
                'Content-Length': str(len(json_data.encode('utf-8')))
            }

            conn.request('POST', self.path, json_data, headers)
            response = conn.getresponse()

            if response.status == 200:
                return True
            else:
                raise RuntimeError(f"HTTP {response.status}: "
                                 f"{response.reason}")

        finally:
            if conn:
                conn.close()

    def send_payload(self, payload: Dict[str, Any]) -> bool:
        """Send JSON payload with retry logic and exponential backoff.

        Args:
            payload: The JSON payload to send

        Returns:
            True if payload was successfully sent, False otherwise
        """
        attempt = 0
        max_attempts = self.retries + 1

        while attempt < max_attempts:
            attempt += 1

            try:
                # Attempt to send the request
                if self._send_request(payload):
                    TimestampedLogger.log(
                        f"[Client] Attempt {attempt}: Success - "
                        f"Status Code 200"
                    )
                    return True

            except socket.timeout:
                error_msg = "TimeoutError"
                TimestampedLogger.log(
                    f"[Client] Attempt {attempt}: {error_msg}"
                )

            except (socket.error, ConnectionRefusedError, OSError):
                error_msg = "ConnectionError"
                TimestampedLogger.log(
                    f"[Client] Attempt {attempt}: {error_msg}"
                )

            except (TypeError, ValueError) as e:
                TimestampedLogger.log(
                    f"[Client] Attempt {attempt}: JSON serialization error: {e}"
                )
                return False  # Don't retry serialization errors

            except Exception as e:
                TimestampedLogger.log(
                    f"[Client] Attempt {attempt}: {type(e).__name__}: {e}"
                )

            # Check if we should retry
            if attempt < max_attempts:
                # Calculate backoff delay with jitter
                base_delay = self.backoff_factor * (2 ** (attempt - 1))
                jitter = random.uniform(0.1, 0.3) * base_delay
                delay = base_delay + jitter

                TimestampedLogger.log(
                    f"[Client] Retrying in {delay:.1f}s"
                )
                time.sleep(delay)
            else:
                TimestampedLogger.log(
                    f"[Client] Attempt {attempt}: Failed - Max retries reached"
                )

        return False


def run_server_mode(args) -> None:
    """Run the application in server mode."""
    server = WebhookServer(
        host=args.host,
        port=args.port,
        drop_probability=args.drop_probability,
        max_delay=args.max_delay
    )

    try:
        server.start(threaded=False)
    except KeyboardInterrupt:
        TimestampedLogger.log("[Server] Received interrupt signal")
    finally:
        server.stop()


def run_client_mode(args) -> None:
    """Run the application in client mode."""
    client = WebhookClient(
        url=args.url,
        retries=args.retries,
        backoff_factor=args.backoff_factor,
        timeout=args.timeout
    )

    # Prepare payload
    payload = {
        "message": args.message,
        "timestamp": time.time(),
        "client_id": "webhook-client-001"
    }

    TimestampedLogger.log(f"[Client] Sending payload to {args.url}")
    TimestampedLogger.log(f"[Client] Retries: {args.retries}, "
                         f"Backoff: {args.backoff_factor}, "
                         f"Timeout: {args.timeout}s")

    success = client.send_payload(payload)

    if success:
        TimestampedLogger.log("[Client] Payload sent successfully")
        sys.exit(0)
    else:
        TimestampedLogger.log("[Client] Failed to send payload")
        sys.exit(1)


def main() -> None:
    """Main entry point for the application."""
    parser = argparse.ArgumentParser(
        description="Webhook Network Simulation System"
    )
    parser.add_argument(
        '--mode',
        choices=['server', 'client'],
        required=True,
        help='Operation mode: server or client'
    )

    # Server arguments
    parser.add_argument(
        '--host',
        default='localhost',
        help='Server host address (default: localhost)'
    )
    parser.add_argument(
        '--port',
        type=int,
        default=8080,
        help='Server port number (default: 8080)'
    )
    parser.add_argument(
        '--drop-probability',
        type=float,
        default=0.3,
        help='Request drop probability 0.0-1.0 (default: 0.3)'
    )
    parser.add_argument(
        '--max-delay',
        type=float,
        default=5.0,
        help='Maximum response delay in seconds (default: 5.0)'
    )

    # Client arguments
    parser.add_argument(
        '--url',
        default='http://localhost:8080',
        help='Webhook URL (default: http://localhost:8080)'
    )
    parser.add_argument(
        '--retries',
        type=int,
        default=3,
        help='Number of retry attempts (default: 3)'
    )
    parser.add_argument(
        '--backoff-factor',
        type=float,
        default=1.0,
        help='Exponential backoff factor (default: 1.0)'
    )
    parser.add_argument(
        '--timeout',
        type=float,
        default=10.0,
        help='Request timeout in seconds (default: 10.0)'
    )
    parser.add_argument(
        '--message',
        default='Hello from webhook client!',
        help='Message to send in payload'
    )

    args = parser.parse_args()

    try:
        if args.mode == 'server':
            run_server_mode(args)
        else:
            run_client_mode(args)

    except ValueError as e:
        TimestampedLogger.log(f"[Error] Configuration error: {e}")
        sys.exit(1)
    except Exception as e:
        TimestampedLogger.log(f"[Error] Unexpected error: {e}")
        sys.exit(1)


if __name__ == '__main__':
    main()


In [ ]:
# tests

"""Unit tests for WebhookClient and WebhookServer."""

import unittest
import threading
import time
from http.client import HTTPConnection
from main import WebhookServer, WebhookClient


class TestWebhookSystem(unittest.TestCase):
    """Tests for WebhookClient and WebhookServer."""

    def setUp(self):
        """Start webhook server on dynamic port."""
        self.server = WebhookServer(
            host='localhost',
            port=0,
            drop_probability=0.0,
            max_delay=0.0
        )
        self.server_thread = threading.Thread(
            target=self.server.start,
            kwargs={'threaded': True},
            daemon=True
        )
        self.server_thread.start()
        max_wait = 50
        wait_count = 0
        while wait_count < max_wait:
            time.sleep(0.1)
            if hasattr(self.server, 'server') and self.server.server:
                break
            wait_count += 1
        if not hasattr(self.server, 'server') or not self.server.server:
            raise RuntimeError("Server failed to start")
        self.port = self.server.server.server_address[1]
        self.base_url = f'http://localhost:{self.port}'

    def tearDown(self):
        """Stop the server after test."""
        self.server.stop()
        time.sleep(0.1)

    def test_successful_delivery(self):
        """Verify successful webhook delivery."""
        client = WebhookClient(self.base_url, 1, 0.1, 2)
        payload = {"message": "test"}
        self.assertTrue(client.send_payload(payload))

    def test_timeout_due_to_delay(self):
        """Verify timeout when all requests are dropped."""
        self.server.stop()
        time.sleep(0.3)
        self.server = WebhookServer('localhost', 0, 1.0, 0.0)
        self.server_thread = threading.Thread(
            target=self.server.start,
            kwargs={'threaded': True},
            daemon=True
        )
        self.server_thread.start()
        max_wait = 50
        wait_count = 0
        while wait_count < max_wait:
            time.sleep(0.1)
            if hasattr(self.server, 'server') and self.server.server:
                break
            wait_count += 1
        port = self.server.server.server_address[1]
        client = WebhookClient(f'http://localhost:{port}', 2, 0.1, 1.0)
        payload = {"message": "timeout"}
        self.assertFalse(client.send_payload(payload))

    def test_all_requests_dropped(self):
        """Verify delivery fails if all requests are dropped."""
        self.server.stop()
        time.sleep(0.3)
        self.server = WebhookServer('localhost', 0, 1.0, 0.0)
        self.server_thread = threading.Thread(
            target=self.server.start,
            kwargs={'threaded': True},
            daemon=True
        )
        self.server_thread.start()
        max_wait = 50
        wait_count = 0
        while wait_count < max_wait:
            time.sleep(0.1)
            if hasattr(self.server, 'server') and self.server.server:
                break
            wait_count += 1
        port = self.server.server.server_address[1]
        client = WebhookClient(f'http://localhost:{port}', 2, 0.1, 1.0)
        payload = {"message": "drop test"}
        self.assertFalse(client.send_payload(payload))

    def test_malformed_payload(self):
        """Verify JSON serialization error on invalid payload."""
        client = WebhookClient(self.base_url, 1, 0.1, 1.0)
        payload = {"bad": set([1, 2, 3])}
        self.assertFalse(client.send_payload(payload))

    def test_invalid_url_format(self):
        """Verify ValueError on malformed URL."""
        with self.assertRaises(ValueError):
            WebhookClient('invalid-url', 1, 1.0, 1.0)

    def test_zero_retries(self):
        """Verify delivery with zero retries."""
        client = WebhookClient(self.base_url, 0, 0.1, 1.0)
        payload = {"key": "value"}
        self.assertTrue(client.send_payload(payload))

    def test_max_delay_boundary(self):
        """Verify delivery within maximum delay limit."""
        self.server.stop()
        time.sleep(0.3)
        self.server = WebhookServer('localhost', 0, 0.0, 10.0)
        self.server_thread = threading.Thread(
            target=self.server.start,
            kwargs={'threaded': True},
            daemon=True
        )
        self.server_thread.start()
        max_wait = 50
        wait_count = 0
        while wait_count < max_wait:
            time.sleep(0.1)
            if hasattr(self.server, 'server') and self.server.server:
                break
            wait_count += 1
        port = self.server.server.server_address[1]
        client = WebhookClient(f'http://localhost:{port}', 1, 0.1, 15.0)
        payload = {"slow": True}
        self.assertTrue(client.send_payload(payload))

    def test_server_shutdown_during_request(self):
        """Verify delivery fails after server shutdown."""
        client = WebhookClient(self.base_url, 1, 0.1, 1.0)
        self.server.stop()
        payload = {"interrupt": True}
        self.assertFalse(client.send_payload(payload))

    def test_invalid_payload_type(self):
        """Verify serialization failure for invalid payload type."""
        client = WebhookClient(self.base_url, 1, 0.1, 1.0)
        payload = {"bad": set([1, 2, 3])}
        self.assertFalse(client.send_payload(payload))

    def test_payload_with_non_serializable_object(self):
        """Verify serialization failure on custom object."""

        class NonSerializable:
            pass

        client = WebhookClient(self.base_url, 1, 0.1, 1.0)
        payload = {"obj": NonSerializable()}
        self.assertFalse(client.send_payload(payload))

    def test_large_payload(self):
        """Verify delivery of large payload."""
        client = WebhookClient(self.base_url, 1, 0.1, 10.0)
        payload = {"data": "x" * 100_000}
        self.assertTrue(client.send_payload(payload))

    def test_multiple_payloads_in_sequence(self):
        """Verify sending multiple sequential payloads."""
        client = WebhookClient(self.base_url, 1, 0.1, 2.0)
        for i in range(5):
            payload = {"sequence": i}
            self.assertTrue(client.send_payload(payload))

    def test_unreachable_port(self):
        """Verify failure for unreachable port."""
        client = WebhookClient('http://localhost:9999', 1, 0.1, 1.0)
        payload = {"fail": True}
        self.assertFalse(client.send_payload(payload))

    def test_server_responds_non_200(self):
        """Verify failure when server is down."""
        self.server.stop()
        time.sleep(0.3)
        client = WebhookClient(self.base_url, 1, 0.1, 1.0)
        payload = {"error": True}
        self.assertFalse(client.send_payload(payload))

    def test_partial_json_failure(self):
        """Verify server handles malformed JSON gracefully."""
        conn = HTTPConnection("localhost", self.port)
        conn.request(
            "POST",
            "/",
            body=b'{"incomplete": ',
            headers={"Content-Type": "application/json"}
        )
        resp = conn.getresponse()
        self.assertEqual(resp.status, 200)

    def test_invalid_method_handling(self):
        """Verify server response to unsupported method."""
        conn = HTTPConnection("localhost", self.port)
        conn.request("GET", "/")
        resp = conn.getresponse()
        self.assertEqual(resp.status, 501)


if __name__ == '__main__':
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/a8b99db2-183c-4b88-be90-4c11ce77c7c1?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F74792%2Fview

#### Model code:

```python
import http.server
import socketserver
import threading
import json
import random
import time
import logging
from http import client
from typing import Dict, Any

# Configure logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(message)s')


class WebhookServer:
    def __init__(self, host: str, port: int, drop_probability: float, max_delay: float):
        self.host = host
        self.port = port
        self.drop_probability = drop_probability
        self.max_delay = max_delay

    def start(self):
        handler = self.make_handler()
        with socketserver.TCPServer((self.host, self.port), handler) as httpd: # <Issue>: socketserver.TCPServer is single-threaded and handles only one request at a time. It should be replaced with socketserver.ThreadingTCPServer to support concurrent delayed requests.
            logging.info(f'Serving on port {self.port}')
            httpd.serve_forever()

    def make_handler(self):
        drop_probability = self.drop_probability
        max_delay = self.max_delay

        class CustomHandler(http.server.BaseHTTPRequestHandler):
            def do_POST(self):
                if random.random() < drop_probability:
                    logging.info('Dropping request (simulated failure)')
                    return

                delay = random.uniform(0, max_delay)
                logging.info(f'Received request. Delaying for {delay:.2f} seconds...')
                time.sleep(delay)

                self.send_response(200)
                self.send_header('Content-type', 'application/json')
                self.end_headers()
                self.wfile.write(json.dumps({"status": "success"}).encode('utf-8'))
                logging.info('Responded with 200 OK')

            def log_message(self, format, *args):
                return  # Suppress default logging

        return CustomHandler


class WebhookClient:
    def __init__(self, url: str, retries: int, backoff_factor: float, timeout: float):
        self.url = url
        self.retries = retries
        self.backoff_factor = backoff_factor
        self.timeout = timeout

    def send_payload(self, payload: Dict[str, Any]):
        parsed_url = client.urlsplit(self.url)
        for attempt in range(1, self.retries + 2):
            try:
                conn = client.HTTPConnection(parsed_url.hostname, parsed_url.port, timeout=self.timeout)
                conn.request('POST', parsed_url.path, body=json.dumps(payload), headers={'Content-Type': 'application/json'}) # <Issue>: If the payload contains non-serializable types (e.g., datetime, bytes), json.dumps() will raise a TypeError, which is not caught. A try/except block should wrap it.
                response = conn.getresponse()

                if response.status == 200:
                    logging.info(f'Attempt {attempt}: Success - Status Code {response.status}')
                    return
                else:
                    logging.info(f'Attempt {attempt}: Failed - Status Code {response.status}')

            except (client.HTTPException, ConnectionError, TimeoutError) as e:
                logging.error(f'Attempt {attempt}: {e.__class__.__name__} - retrying in {self.backoff_factor * (2 ** (attempt - 1)):.1f}s')

            # Exponential backoff with jitter
            time.sleep(self.backoff_factor * (2 ** (attempt - 1)) * (0.5 + 0.5 * random.random()))

        logging.error('Max retries reached, giving up.') # <Issue>: While a final message is logged, no summary or detail is included. A clear indication of total attempts and failure reasons should be logged for observability.


def main():
    import argparse
    parser = argparse.ArgumentParser(description='Simulate network conditions for webhook communication')
    parser.add_argument('--mode', choices=['server', 'client'], required=True, help='Run mode: server or client')

    args = parser.parse_args()

    if args.mode == 'server':
        server = WebhookServer(host='localhost', port=8080, drop_probability=0.3, max_delay=5.0) # <Issue>: port, drop_probability, and max_delay are hardcoded instead of being passed from command-line arguments, making testing less flexible.
        server.start()
    elif args.mode == 'client':
        client = WebhookClient(url='http://localhost:8080', retries=3, backoff_factor=1.0, timeout=2.0)
        payload = {"message": "Hello, Webhook!"}
        client.send_payload(payload)


if __name__ == '__main__':
    main()
```